In [1]:
from datasets import load_dataset

dataset = load_dataset("wikitext", "wikitext-103-v1", split="train")


DATASET_NAME = 'adsabs/WIESP2022-NER'
testset = load_dataset(DATASET_NAME, split='test')
trainset = load_dataset(DATASET_NAME, split='train')

In [2]:
unique_tags = set(tag for example in testset["ner_tags"] for tag in example)
sorted_tags = sorted(unique_tags - {"O"})  # Exclude 'O' from sorted tags
list_tags = list(unique_tags)

clear = [tag[2:] for tag in list_tags]
labels_list = list(dict.fromkeys(clear))
labels_string = ' '.join(labels_list)

In [13]:
def split_text_into_chunks(text, chunk_size=500):
    words = text.split()
    for i in range(0, len(words), chunk_size):
        yield " ".join(words[i:i + chunk_size])

In [14]:
chunk_size = 256
text_chunks = []
for example in dataset:
    text_chunks.extend(list(split_text_into_chunks(example["text"], chunk_size=chunk_size)))

# Check a few chunks
print(text_chunks[:3])  # Display the first 3 chunks

['= Valkyria Chronicles III =', 'Senjō no Valkyria 3 : <unk> Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " <unk> Raven " .', "The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game more forgiving for series newcomers

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the Qwen model and tokenizer
model_name = "Qwen/Qwen2.5-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
def model_generate(model, tokenizer, prompt, mt=1024):
    model_inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=4096).to(model.device)
    generated_ids = model.generate(**model_inputs, max_new_tokens=mt)
    generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response

In [23]:
import json
import re

def extract_entities_from_output(model_output):
    """
    Extract the JSON content from the model output and reformat it.
    """
    try:
        # Extract JSON portion from the output
        match = re.search(r"\{.*\}", model_output, re.DOTALL)
        if match:
            entities_json = json.loads(match.group(0))  # Parse JSON
            entities = entities_json.get("entities", [])  # Get "entities" list

            # Reformat the entities
            formatted_entities = [
                {"text": entity["entity"], "label": entity["label"]}
                for entity in entities
            ]
            return formatted_entities
    except (json.JSONDecodeError, KeyError) as e:
        print(f"Error processing output: {e}")
    return []

In [18]:
short_text_chunks = text_chunks[:10000]

NameError: name 'text_chunks' is not defined

In [ ]:
all_results = []

for i, chunk in enumerate(short_text_chunks):
    try:
        print(f"Processing chunk {i + 1}/{len(short_text_chunks)}...")
        prompt = f"Perform Named Entity Recognition (NER) on the following text:\n\n{chunk}\n\nWith given labels: \n\n{labels}\n\nReturn the results in JSON format." 
        prompt += """ Write nothing except of JSON, format looks like this {"entities": [{"entity": "Sarrah","label": "Person"}]}"""
        ner_result = model_generate(chunk, labels_string, model, tokenizer, promt)
        entities = extract_entities_from_output(ner_result)
        all_results.append(entities)
    except Exception as e:
            print(f"Error processing chunk {i + 1}: {e}")

Processing chunk 1/10000...
Processing chunk 2/10000...
Processing chunk 3/10000...
Processing chunk 4/10000...
Processing chunk 5/10000...
Processing chunk 6/10000...
Processing chunk 7/10000...
Processing chunk 8/10000...
Processing chunk 9/10000...
Error processing output: Expecting ',' delimiter: line 127 column 48 (char 6242)
Processing chunk 10/10000...
Processing chunk 11/10000...
Processing chunk 12/10000...
Processing chunk 13/10000...
Error processing output: Expecting ',' delimiter: line 138 column 47 (char 6789)
Processing chunk 14/10000...
Processing chunk 15/10000...
Processing chunk 16/10000...
Processing chunk 17/10000...
Processing chunk 18/10000...
Processing chunk 19/10000...
Processing chunk 20/10000...
Processing chunk 21/10000...
Processing chunk 22/10000...
Processing chunk 23/10000...
Processing chunk 24/10000...
Processing chunk 25/10000...
Processing chunk 26/10000...
Processing chunk 27/10000...
Processing chunk 28/10000...
Processing chunk 29/10000...
Proces

In [10]:
output_file = "ner_dataset_10k.json"
with open(output_file, "w") as f:
        json.dump(all_results, f, indent=4)

In [10]:
output_file = "ner_dataset_10k.json"
with open(output_file, "r") as file:
    ner_data = json.load(file)

print(ner_data[:3])

[[], [{'text': 'Gallia', 'label': 'Location'}, {'text': 'Imperial', 'label': 'Organization'}, {'text': 'Second Europan War', 'label': 'CelestialRegion'}, {'text': 'Valkyria Chronicles III', 'label': 'Identifier'}, {'text': 'Sega', 'label': 'Company'}, {'text': 'Media.Vision', 'label': 'Company'}, {'text': 'PlayStation Portable', 'label': 'Device'}, {'text': 'Japan', 'label': 'Location'}, {'text': 'Second Europan War', 'label': 'Event'}, {'text': 'Gallia', 'label': 'EntityOfFutureInterest'}], [{'text': 'Honjou', 'label': 'Person'}, {'text': 'Ozawa', 'label': 'Person'}, {'text': "May 'n.", 'label': 'EntityOfFutureInterest'}]]


In [17]:
dist_set = []
for text, entities in zip(short_text_chunks, ner_data):
     dist_set.append({"text": text, "entities": entities})

In [18]:
print(dist_set[1])

{'text': 'Senjō no Valkyria 3 : <unk> Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " <unk> Raven " .', 'entities': [{'text': 'Gallia', 'label': 'Location'}, {'text': 'Imperial', 'label': 'Organization'}, {'text': 'Second Europan War', 'label': 'CelestialRegion'}, {'text': 'Valkyria Chronicles III', 'label': 'Identifier'}, {'text': 'Sega', 'label': 'Company'}, {'text': 'Media.Vision', 'label':

In [19]:
file_name = "ner_dist_set_10k.json"
with open(file_name, "w", encoding="utf-8") as f:
        json.dump(dist_set, f, indent=4)

In [20]:
import json
output_file = "ner_dist_set_10k.json"
with open(output_file, "r") as file:
    dist_dataset = json.load(file)
print(dist_dataset[1])

{'text': 'Senjō no Valkyria 3 : <unk> Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " <unk> Raven " .', 'entities': [{'text': 'Gallia', 'label': 'Location'}, {'text': 'Imperial', 'label': 'Organization'}, {'text': 'Second Europan War', 'label': 'CelestialRegion'}, {'text': 'Valkyria Chronicles III', 'label': 'Identifier'}, {'text': 'Sega', 'label': 'Company'}, {'text': 'Media.Vision', 'label':

In [5]:
def parse_output(output_string, classes):
    for cls in classes:
        if cls in output_string:
            return cls
    return "not found"

In [6]:
from collections import defaultdict
from tqdm import tqdm

def validate(model, dataset, classes):
    true_positives = defaultdict(int)
    false_positives = defaultdict(int)
    false_negatives = defaultdict(int)
    fine_tune_data = []

    for subset in tqdm(dataset, total=len(dataset)):
        chunk = subset["text"]
        ner_chunk = subset["entities"]
        if ner_chunk == []:
            continue
        for entity in ner_chunk:
            text = entity["text"]
            label = entity["label"]
            prompt = f"Perform Named Entity Recognition (NER) for one word. you should tell what class describes suggested word best. There is text: \n\n'{chunk}'.\n\nHere is the list of classes: {classes}.\n Suggested word to clasify: '{text}'\nJust answer with a name of a class of a suggested word. If you'll answer incorrectly your mother would die"
            prediction = model_generate(model, tokenizer, prompt, mt=32)
            pred = parse_output(prediction, classes)
            #tqdm.write(f"pr: {prompt}, \nmodel said: {prediction}\n\n")
            #tqdm.write(f"gt: {label}, predicted: {pred}\n\n\n")
            if pred == label:
                true_positives[label] += 1
            elif pred == "not found":
                false_negatives[label] += 1
            else:
                false_positives[pred] += 1
                false_negatives[label] += 1

    metrics = {}
    for cls in classes:
        tp = true_positives[cls]
        fp = false_positives[cls]
        fn = false_negatives[cls]

        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        metrics[cls] = {
            "precision": round(precision, 4),
            "recall": round(recall, 4),
            "f1_score": round(f1_score, 4),
            "true_positives": tp,
            "false_positives": fp,
            "false_negatives": fn,
        }

    # Summarize overall metrics
    overall_tp = sum(true_positives.values())
    overall_fp = sum(false_positives.values())
    overall_fn = sum(false_negatives.values())

    overall_precision = overall_tp / (overall_tp + overall_fp) if (overall_tp + overall_fp) > 0 else 0
    overall_recall = overall_tp / (overall_tp + overall_fn) if (overall_tp + overall_fn) > 0 else 0
    overall_f1_score = (
        2 * (overall_precision * overall_recall) / (overall_precision + overall_recall)
        if (overall_precision + overall_recall) > 0
        else 0
    )

    report = {
        "class_metrics": metrics,
        "overall_metrics": {
            "precision": round(overall_precision, 4),
            "recall": round(overall_recall, 4),
            "f1_score": round(overall_f1_score, 4),
        },
    }
    return report

In [65]:
subsetik = 
#labels_list.remove('')
print(val_json[2], val_chunks[2])
results = validate(model, dist_dataset, labels_list)
print(results)

[{'text': 'Honjou', 'label': 'Person'}, {'text': 'Ozawa', 'label': 'Person'}, {'text': "May 'n.", 'label': 'EntityOfFutureInterest'}] The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game more forgiving for series newcomers . Character designer <unk> Honjou and composer Hitoshi Sakimoto both returned from previous entries , along with Valkyria Chronicles II director Takeshi Ozawa . A large team of writers handled the script . The game 's opening theme was sung by May 'n .


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [17:49<00:00, 10.70s/it]

{'class_metrics': {'TextGarbage': {'precision': 0.1961, 'recall': 0.1515, 'f1_score': 0.1709, 'true_positives': 10, 'false_positives': 41, 'false_negatives': 56}, 'ComputingFacility': {'precision': 0.1667, 'recall': 0.4, 'f1_score': 0.2353, 'true_positives': 2, 'false_positives': 10, 'false_negatives': 3}, 'Organization': {'precision': 0.7308, 'recall': 0.6032, 'f1_score': 0.6609, 'true_positives': 38, 'false_positives': 14, 'false_negatives': 25}, 'Citation': {'precision': 0.0, 'recall': 0, 'f1_score': 0, 'true_positives': 0, 'false_positives': 1, 'false_negatives': 0}, 'Observatory': {'precision': 0.75, 'recall': 0.25, 'f1_score': 0.375, 'true_positives': 3, 'false_positives': 1, 'false_negatives': 9}, 'Survey': {'precision': 1.0, 'recall': 1.0, 'f1_score': 1.0, 'true_positives': 2, 'false_positives': 0, 'false_negatives': 0}, 'Instrument': {'precision': 0.3333, 'recall': 0.5714, 'f1_score': 0.4211, 'true_positives': 4, 'false_positives': 8, 'false_negatives': 3}, 'Person': {'precisi

In [7]:
from nltk.tokenize.treebank import TreebankWordDetokenizer

def transform_ner_dataset(dataset):
    transformed_dataset = []
    for sample in dataset:
        sample_dataset = []
        # Extract the text and entities
        text = TreebankWordDetokenizer().detokenize(sample["tokens"])
        entities = sample["ner_tags"]
        tokens = sample["tokens"]
        entity_list = []

        current_entity = {"text": "", "label": ""}
        inside_entity = False

        for token, label in zip(tokens, entities):
            if label.startswith("B-"):  # Begin a new entity
                if inside_entity:
                    entity_list.append(current_entity)
                current_entity = {"text": token, "label": label[2:]}
                inside_entity = True
            elif label.startswith("I-") and inside_entity:  # Continue the current entity
                current_entity["text"] += f" {token}"
            else:  # Outside any entity
                if inside_entity:
                    entity_list.append(current_entity)
                inside_entity = False

        # Add any remaining entity
        if inside_entity:
            entity_list.append(current_entity)

        # Append the transformed result for this chunk
        transformed_dataset.append({"text": text, "entities": entity_list})

    return transformed_dataset

In [8]:
transformed_train = transform_ner_dataset(trainset)
transformed_test = transform_ner_dataset(testset)

In [9]:
results = validate(model, transformed_test[:100], labels_list) # 0.57 - f1 qwen2.5 - 3b
print(results)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [40:47<00:00, 24.48s/it]

{'class_metrics': {'Dataset': {'precision': 0.0, 'recall': 0.0, 'f1_score': 0, 'true_positives': 0, 'false_positives': 83, 'false_negatives': 8}, 'URL': {'precision': 0.8421, 'recall': 0.5926, 'f1_score': 0.6957, 'true_positives': 16, 'false_positives': 3, 'false_negatives': 11}, 'Telescope': {'precision': 0.4407, 'recall': 0.2955, 'f1_score': 0.3537, 'true_positives': 26, 'false_positives': 33, 'false_negatives': 62}, 'CelestialObject': {'precision': 0.5, 'recall': 0.3333, 'f1_score': 0.4, 'true_positives': 2, 'false_positives': 2, 'false_negatives': 4}, 'Person': {'precision': 0.9512, 'recall': 0.858, 'f1_score': 0.9022, 'true_positives': 429, 'false_positives': 22, 'false_negatives': 71}, 'Collaboration': {'precision': 0.1404, 'recall': 0.64, 'f1_score': 0.2302, 'true_positives': 16, 'false_positives': 98, 'false_negatives': 9}, 'Grant': {'precision': 0.7246, 'recall': 0.5696, 'f1_score': 0.6378, 'true_positives': 221, 'false_positives': 84, 'false_negatives': 167}, 'Citation': {'pr

In [25]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the Qwen model and tokenizer
model_name = "Qwen/Qwen2.5-0.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

In [26]:
results = validate(model, transformed_test[:100], labels_list) # 0.13 - f1 qwen2.5 - 0.5B
print(results)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [39:26<00:00, 23.66s/it]

{'class_metrics': {'Survey': {'precision': 0.0847, 'recall': 0.3947, 'f1_score': 0.1395, 'true_positives': 15, 'false_positives': 162, 'false_negatives': 23}, 'Software': {'precision': 0.0874, 'recall': 0.2105, 'f1_score': 0.1236, 'true_positives': 16, 'false_positives': 167, 'false_negatives': 60}, 'Citation': {'precision': 0.0, 'recall': 0.0, 'f1_score': 0, 'true_positives': 0, 'false_positives': 43, 'false_negatives': 36}, 'Formula': {'precision': 0.0, 'recall': 0.0, 'f1_score': 0, 'true_positives': 0, 'false_positives': 2, 'false_negatives': 1}, 'Observatory': {'precision': 0.2685, 'recall': 0.4167, 'f1_score': 0.3265, 'true_positives': 40, 'false_positives': 109, 'false_negatives': 56}, 'CelestialObjectRegion': {'precision': 0.0, 'recall': 0, 'f1_score': 0, 'true_positives': 0, 'false_positives': 11, 'false_negatives': 0}, 'EntityOfFutureInterest': {'precision': 0.0, 'recall': 0.0, 'f1_score': 0, 'true_positives': 0, 'false_positives': 89, 'false_negatives': 1}, 'Organization': {'

In [ ]:
from transformers import AutoTokenizer

def tokenize_dataset(dataset, tokenizer, max_length=512):
    """
    Tokenizes the dataset for fine-tuning.
    """
    tokenized_data = []
    for example in dataset:
        # Combine prompt and answer
        input_text =  f"Perform Named Entity Recognition (NER) for one word. you should tell what class describes suggested word best. There is text: \n\n'{chunk}'.\n\nHere is the list of classes: {classes}.\n Suggested word to clasify: '{text}'\nJust answer with a name of a class of a suggested word."
        tokenized = tokenizer(
            input_text,
            max_length=max_length,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )
        tokenized_data.append({
            "input_ids": tokenized["input_ids"].squeeze(0),
            "attention_mask": tokenized["attention_mask"].squeeze(0)
        })
    return tokenized_data